## EXPLORATORY DATA ANALYSIS

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
# connecting database connection
conn=sqlite3.connect('inventory.db')

In [ ]:
# checking tables present in the database
tables=pd.read_sql_query("SELECT name FROM sqlite_master WHERE TYPE ='table'",conn)
tables

In [ ]:
pd.read_sql("select count (*) from purchases",conn)

In [ ]:
for table in tables['name']:
    print('-' * 50, f'{table}', '_' * 50)
    
    count_query = f"SELECT COUNT(*) as count FROM {table}"
    count = pd.read_sql(count_query, conn)['count'].values[0]
    print(f'Count of records: {count}')
    
    display(pd.read_sql(f"SELECT * FROM {table} limit 5", conn))


In [ ]:
for table in tables['name']:
    df = pd.read_sql(f"SELECT * FROM {table} LIMIT 1", conn)  # Get one row
    print(f"Table: {table}")
    print("Columns:", df.columns.tolist())
    print('-' * 50)

### Analysis 4466 vendor using the all tables

In [ ]:
purchases=pd.read_sql_query("select*from purchases where VendorNumber=4466",conn)
purchases

In [ ]:
purchases_prices=pd.read_sql_query("select * from purchase_prices where VendorNumber = 4466",conn)
purchases_prices

In [ ]:
sales=pd.read_sql_query("select * from sales where VendorNo = 4466",conn)
sales

In [ ]:
vendor_invoice=pd.read_sql_query("select*from vendor_invoice where VendorNumber=4466",conn)
vendor_invoice

In [ ]:
purchases.groupby(['Brand','PurchasePrice'])(['Quantity','Dollars']).sum()

In [ ]:
purchases.groupby(['Brand', 'PurchasePrice'])[['Quantity', 'Dollars']].sum()


In [ ]:
purchases_prices

### using sales datasets

In [ ]:
sales.groupby(['Brand'])[['SalesDollars','SalesPrice','SalesQuantity']].sum()

### comparision purchases and sales

In [ ]:
# Step 1: Aggregate sales data
sales_summary = sales.groupby('Brand')[['SalesDollars', 'SalesPrice', 'SalesQuantity']].sum().reset_index()

# Step 2: Aggregate purchases data
purchase_summary = purchases.groupby('Brand')[['Dollars', 'Quantity']].sum().reset_index()

# Step 3: Merge the two summaries on 'Brand'
comparison = pd.merge(sales_summary, purchase_summary, on='Brand', how='outer')

# Display the comparison
print(comparison)

### FINDING THE SUM OF FREIGHT COST ACCORDING TO VENDOR NUMBER

In [ ]:
freight_summary=pd.read_sql_query("""select VendorNumber,SUM(Freight) as FreightCost 
from vendor_invoice
Group BY VendorName""",conn)

In [ ]:
freight_summary

### summary from table purchases and from purchases_price table

In [ ]:
pd.read_sql_query("""
SELECT 
    p.VendorNumber,
    p.VendorName,
    p.Brand,
    p.PurchasePrice,
    pp.Volume,
    pp.Price as ActualPrice,
    SUM(p.Quantity) as TotalPurchaseQuantity,
    SUM(p.Dollars) as TotalPurchaseDollars
FROM purchases p
JOIN purchase_prices pp
    ON p.Brand = pp.Brand
WHERE p.PurchasePrice > 0
GROUP BY p.VendorNumber, p.VendorName, p.Brand
ORDER BY TotalPurchaseDollars
""", conn)


when we remove the condition Where p.purchasePrice >0 it give o/p having o purchasePrise but it is not possible may be data is incorrect

### summary from Sales Table

In [ ]:
Sales = pd.read_sql_query("SELECT * FROM sales", conn)


In [ ]:
Sales.columns


In [ ]:
pd.read_sql_query("""
SELECT 
    VendorNo,
    Brand,
    SUM(SalesDollars) as TotalSalesDollars,
    SUM(SalesPrice) as TotalSalesPrice,
    SUM(SalesQuantity) as TotalSalesQuantity,
    SUM(ExciseTax) as TotalExciseTax
FROM sales
GROUP BY VendorNo, Brand
ORDER BY TotalSalesDollars
""", conn)


### We will join the above three table by using join

In [ ]:
## query optimization is very issential

In [ ]:
vendor_sales_summary = pd.read_sql_query("""
WITH FreightSummary AS (
    SELECT
        VendorNumber,
        SUM(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
),
PurchaseSummary AS (
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        p.PurchasePrice,
        pp.Price AS ActualPrice,
        pp.Volume,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
        ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
),
SalesSummary AS (
    SELECT
        VendorNo,
        Brand,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(SalesPrice) AS TotalSalesPrice,
        SUM(ExciseTax) AS TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
)
SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo
    AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC
""", conn)


In [ ]:
vendor_sales_summary

#### After creating summary data we will check for cleaning and other EDA work

In [ ]:
vendor_sales_summary.info()


In [ ]:
vendor_sales_summary.isnull().sum()

In [ ]:
vendor_sales_summary.head()

In [ ]:
vendor_sales_summary['VendorName'].unique()

In [ ]:
vendor_sales_summary['Description'].unique()

#### we find three inconsistance in our vendor_sales_summary data set 
#1.volume data type is object instead of int
#2.white spapaes in vendor name
#3.null values (It occurred because the product was not purchased by anyone.)

In [ ]:
vendor_sales_summary['Volume']=vendor_sales_summary['Volume'].astype('float64')

In [ ]:
vendor_sales_summary.fillna(0 ,inplace=True)

In [ ]:
vendor_sales_summary['VendorName']=vendor_sales_summary['VendorName'].str.strip()

In [ ]:
vendor_sales_summary.isnull().sum()

In [ ]:
vendor_sales_summary['VendorName'].unique()

In [ ]:
vendor_sales_summary.info()

### now we will create more columns with new features like:
#1.gross margin 
#2.Stock turnover
#3.gross profit,etc

### 1.Gross Profit ( Sales price- Actual Price/purchase price)in dollers

In [ ]:
vendor_sales_summary['GrossProfit']=vendor_sales_summary['TotalSalesDollars']-vendor_sales_summary['TotalPurchaseDollars']

In [ ]:
vendor_sales_summary

### Calculating Progit Margin in Percentage

In [ ]:
vendor_sales_summary['ProfitMargin']=(vendor_sales_summary['GrossProfit']/vendor_sales_summary['TotalSalesDollars'])*100

In [ ]:
vendor_sales_summary.head()

### Calcutating Stock Turnover 
#when a vendor purchase a product all sells all the quantity of the product then stock turnover is 1 other wise less than 1
#if it is greater than 1 then vendors have previous stock which they are selling with the new quantity

In [ ]:
vendor_sales_summary['StockTurnover']=vendor_sales_summary['TotalSalesQuantity']/vendor_sales_summary['TotalPurchaseQuantity']

In [ ]:
vendor_sales_summary['SalestoPurchaseRatio']=vendor_sales_summary['TotalSalesDollars']/vendor_sales_summary['TotalPurchaseQuantity']

In [ ]:
vendor_sales_summary.head()

## we will sane vendor_sales_summary this data in a database:
#it can be possible by 2 methods 
#1.by sql query
#2.by creating an empty table (used in companies)

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE vendor_sales_summary (
    VendorNumber INT,
    VendorName VARCHAR(100),
    Brand INT,
    Description VARCHAR(100),
    PurchasePrice DECIMAL(10,2),
    ActualPrice DECIMAL(10,2),
    Volume INT,
    TotalPurchaseQuantity INT,
    TotalPurchaseDollars DECIMAL(15,2),
    TotalSalesQuantity INT,
    TotalSalesDollars DECIMAL(15,2),
    TotalSalesPrice DECIMAL(15,2),
    TotalExciseTax DECIMAL(15,2),
    FreightCost DECIMAL(15,2),
    GrossProfit DECIMAL(15,2),
    ProfitMargin DECIMAL(15,2),
    StockTurnover DECIMAL(15,2),
    SalesToPurchaseRatio DECIMAL(15,2),
    PRIMARY KEY (VendorNumber, Brand)
)
""")


In [ ]:
vendor_sales_summary.to_sql('vendor_sales_summary',conn,if_exists='replace',index=False)

In [ ]:
pd.read_sql_query("select* from vendor_sales_summary",conn)

In [ ]:
import pandas as pd

data.to_csv('vendor_sales_summary', index=False)
